In [1]:
import pandas as pd
import os
from IPython.display import display
import sys
# from tabulate import tabulate

pd.set_option('mode.chained_assignment', None)

In [2]:
initial_amount = 500
months_to_distribute_initial_amount = 5
term_months = 10
term_holidays = 2
credit_rate_yearly = 0.12
total_time_in_months = 24 #120

credit_rate_monthly = credit_rate_yearly/12
initial_payment = initial_amount/months_to_distribute_initial_amount

In [3]:
cl_dct = {}
col_names_basic = ["Term","Amount_left"]
col_names_clients = ["Paid","Return_principal","Return_interest"]

cl_data_ini = []
for i in range(total_time_in_months+1):
  cl_data_ini.append(("m%03d"%(i+1),None))
df = pd.DataFrame(cl_data_ini,columns=col_names_basic)


amount_left = initial_amount
amount_available = [amount_left]

for clt in range(total_time_in_months):#-term_months):
  client_number = clt+1
  
  cl_data_ini = []
  if client_number <= total_time_in_months-term_months+1:
    for i in range(total_time_in_months+1):
      if i+1 < client_number:
        cl_data_ini.append(("m%03d"%(i+1),None,0.0,0.0,0.0))
      elif i+1 == client_number:
        payment = initial_payment if initial_payment <= amount_left else amount_left
        amount_left -= payment
        return_principal = payment/(term_months-term_holidays)
        principal_left = payment
        cl_data_ini.append(("m%03d"%(i+1),amount_left,payment,0.0,0.0))

      elif i < client_number + term_holidays:
        cl_data_ini.append(("m%03d"%(i+1),amount_left,payment,0.0,payment*credit_rate_monthly))
      elif i < client_number + term_months:
        cl_data_ini.append(("m%03d"%(i+1),None,0.0,return_principal,payment*credit_rate_monthly))
      else:
        cl_data_ini.append(("m%03d"%(i+1),None,0.0,0.0,0.0))

    df0 = pd.DataFrame(cl_data_ini,columns=col_names_basic + list(map(lambda x:x+"_%03d"%client_number,col_names_clients)))  
    df0[col_names_basic[-1]][:client_number-1] = df[col_names_basic[-1]][:client_number-1]
    df = df.drop(col_names_basic[-1],axis=1,).merge(df0,on=col_names_basic[0],sort=True)

    cols = list(df)
    cols.insert(1,cols.pop(cols.index(col_names_basic[-1])))
    df = df[cols]
    
    amount_left += df.loc[client_number,[col for col in df if col.startswith(col_names_clients[1])]].sum()
  else:
    df[col_names_basic[-1]][client_number-1] = amount_left  
    amount_left += df.loc[client_number,[col for col in df if col.startswith(col_names_clients[1])]].sum()
    
    df[col_names_basic[-1]][client_number] = amount_left
    
  amount_available.append(amount_left)
#   print("month_%02d: %0.2f$"%(client_number+1,amount_left))
  
# display(df)
df["Total_return_interest"] = df[[col for col in df if col.startswith(col_names_clients[2])]].sum(axis=1)
df[["Total_cumul_return_interest"]] = df[["Total_return_interest"]].cumsum()
df["Amount_available"] = amount_available

df_result = df[["Term"]+[col for col in df if col.startswith(("Amount","Total_"))]]
df_result = df_result[['Term','Amount_available', 'Amount_left', 'Total_return_interest', 'Total_cumul_return_interest']]
df_result[df_result.columns[1:]] = df_result[df_result.columns[1:]].applymap('{:,.02f}'.format)

display(df_result)

,Term,Amount_available,Amount_left,Total_return_interest,Total_cumul_return_interest
0,m001,500.00,400.00,0.00,0.00
1,m002,400.00,300.00,1.00,1.00
2,m003,300.00,200.00,2.00,3.00
3,m004,212.50,112.50,3.00,6.00
4,m005,137.50,37.50,4.00,10.00
5,m006,75.00,0.00,5.00,15.00
6,m007,50.00,0.00,5.75,20.75
7,m008,62.50,0.00,6.25,27.00
8,m009,71.88,0.00,6.88,33.88
9,m010,78.12,0.00,7.59,41.47


In [4]:
# Calculation explanation
display(df)

,Term,Amount_left,Paid_001,Return_principal_001,Return_interest_001,Paid_002,Return_principal_002,Return_interest_002,Paid_003,Return_principal_003,...,Return_interest_013,Paid_014,Return_principal_014,Return_interest_014,Paid_015,Return_principal_015,Return_interest_015,Total_return_interest,Total_cumul_return_interest,Amount_available
0,m001,400,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,500.000000
1,m002,300,100.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,400.000000
2,m003,200,100.0,0.0,1.0,100.0,0.0,1.0,100.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,3.000000,300.000000
3,m004,112.5,0.0,12.5,1.0,100.0,0.0,1.0,100.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,6.000000,212.500000
4,m005,37.5,0.0,12.5,1.0,0.0,12.5,1.0,100.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,10.000000,137.500000
5,m006,0,0.0,12.5,1.0,0.0,12.5,1.0,0.0,12.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,15.000000,75.000000
6,m007,0,0.0,12.5,1.0,0.0,12.5,1.0,0.0,12.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.750000,20.750000,50.000000
7,m008,0,0.0,12.5,1.0,0.0,12.5,1.0,0.0,12.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.250000,27.000000,62.500000
8,m009,0,0.0,12.5,1.0,0.0,12.5,1.0,0.0,12.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.875000,33.875000,71.875000
9,m010,0,0.0,12.5,1.0,0.0,12.5,1.0,0.0,12.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.593750,41.468750,78.125000
